https://github.com/timcnicholls/transport_api_demo/blob/master/harwell_wantage_bus.py

https://github.com/timcnicholls/home-assistant/blob/transport-api/homeassistant/components/sensor/uk_transport.py

In [21]:
import requests
import json
import re
import pprint
from datetime import datetime, timedelta

api_secrets_file = 'transportapi_secrets_example.json'

transport_api_url_base = "https://transportapi.com/v3/uk/"

def print_json(json_data):
    pprint.PrettyPrinter().pprint(json_data)

def load_api_secrets(filename):
    try:
        with open(filename, 'r') as fp:
            api_params = json.load(fp)
    except Exception as e:
        print('Failed to load API secrets key: {}'.format(e))
        api_params = None
    return api_params

def _delta_mins(hhmm_time_str):
    """Calculate time delta in minutes to a time in hh:mm format."""
    now = datetime.now()
    hhmm_time = datetime.strptime(hhmm_time_str, '%H:%M')

    hhmm_datetime = datetime(
        now.year, now.month, now.day,
        hour=hhmm_time.hour, minute=hhmm_time.minute
    )
    if hhmm_datetime < now:
        hhmm_datetime += timedelta(days=1)

    delta_mins = (hhmm_datetime - now).seconds // 60
    return delta_mins

In [22]:
api_params = load_api_secrets(api_secrets_file)

In [136]:
# Tims code
ATTR_STOP_NAME = 'stop_name'
ATTR_REQUEST_TIME = 'request_time'
ATTRIBUTION = "Data provided by transportapi.com"
ATTR_ATTRIBUTION = 'attribution'  # from const module

class UkTransportSensor():  # Entity
    """
    Sensor that reads the UK transport web API.
    transportapi.com provides comprehensive transport data for UK train, tube
    and bus travel across the UK via simple JSON API. Subclasses of this
    base class can be used to access specific types of information.
    """

    TRANSPORT_API_URL_BASE = "https://transportapi.com/v3/uk/"
    ICON = 'mdi:car'

    def __init__(self, name, api_app_id, api_app_key, url):
        """Initialize the sensor."""
        self._data = {}    # dict to place all the json data in
        self._api_app_id = api_app_id
        self._api_app_key = api_app_key
        self._url = self.TRANSPORT_API_URL_BASE + url
        self._name = name
        self._state = None

    @property
    def name(self):
        """Return the name of the sensor."""
        return self._name

    @property
    def state(self):
        """Return the state of the sensor."""
        return self._state

    @property
    def icon(self):
        """Icon to use in the frontend, if any."""
        return self.ICON

    def _do_api_request(self, params):
        """Perform an API request."""
        request_params = dict({
            'app_id': self._api_app_id,
            'app_key': self._api_app_key,
        }, **params)
        try:
            response = requests.get(self._url, params=request_params)
            response.raise_for_status()
            self._data = response.json()
        except requests.RequestException as req_exc:
            _LOGGER.warning(
                'Invalid response from transportapi.com: %s', req_exc
            )

In [137]:
ATTR_NEXT_BUSES = 'next_buses'
ATTR_ATCOCODE = 'atcocode'
ATTR_LOCALITY = 'locality'

class UkTransportLiveBusTimeSensor(UkTransportSensor):
    """Live bus time sensor from UK transportapi.com."""
    ICON = 'mdi:bus'

    def __init__(self, api_app_id, api_app_key, stop_atcocode, bus_direction):
        """Construct a live bus time sensor."""
        self._stop_atcocode = stop_atcocode
        self._bus_direction = bus_direction
        self._next_buses = []
        self._destination_re = re.compile(
            '{}'.format(bus_direction), re.IGNORECASE
        )

        sensor_name = 'Next bus to {}'.format(bus_direction)
        stop_url = 'bus/stop/{}/live.json'.format(stop_atcocode)

        UkTransportSensor.__init__(
            self, sensor_name, api_app_id, api_app_key, stop_url
        )

    def update(self):
        """Get the latest live departure data for the specified stop."""
        params = {'group': 'route', 'nextbuses': 'no'}

        self._do_api_request(params)

        if self._data != {}:
            self._next_buses = []

            for (route, departures) in self._data['departures'].items():
                for departure in departures:
                    if self._destination_re.search(departure['direction']):
                        self._next_buses.append({
                            'route': route,
                            'direction': departure['direction'],
                            'scheduled': departure['aimed_departure_time'],
                            'estimated': departure['best_departure_estimate']
                        })

            self._state = min(map(
                _delta_mins, [bus['scheduled'] for bus in self._next_buses]
            ))

    @property
    def device_state_attributes(self):
        """Return other details about the sensor state."""
        if self._data is not None:
            attrs = {ATTR_ATTRIBUTION: ATTRIBUTION}  # {'attribution': 'Data provided by transportapi.com'}
            for key in [
                    ATTR_ATCOCODE, ATTR_LOCALITY, ATTR_STOP_NAME,
                    ATTR_REQUEST_TIME
            ]:
                attrs[key] = self._data.get(key)           # place these attributes 
            attrs[ATTR_NEXT_BUSES] = self._next_buses      # not in 
            print_json(attrs)
            return attrs

    @property
    def unit_of_measurement(self):
        """Return the unit this state is expressed in."""
        return "min"

In [138]:
CONF_API_APP_ID = api_params['app_id']
CONF_API_APP_KEY = api_params['app_key']
bus_stop_atcocode =  '340000368SHE'
bus_direction =  'Wantage'

Bus_sensor = UkTransportLiveBusTimeSensor(CONF_API_APP_ID, CONF_API_APP_KEY, bus_stop_atcocode, bus_direction)
Bus_sensor.update()

In [139]:
for (route, departures) in Bus_sensor._data['departures'].items():
    print(route)

X32
34
32A


In [140]:
Bus_sensor._data.get('next_buses') 

In [141]:
# As per bus but route becomes origin_name, direction becomes destination_name, next_suses becomes next_trains

class UkTransportLiveTrainTimeSensor(UkTransportSensor):
    """Live train time sensor from UK transportapi.com."""
    ICON = 'mdi:train'

    def __init__(self, api_app_id, api_app_key, station_code, destination_name):
        """Construct a live bus time sensor."""
        self._station_code = station_code         # stick to the naming convention of transportAPI
        self._destination_name = destination_name
        self._next_trains = {}
        self._destination_re = re.compile(
            '{}'.format(destination_name), re.IGNORECASE
        )

        sensor_name = 'Next train to {}'.format(destination_name)
        query_url =  'train/station/{}/live.json'.format(station_code)
        
        print(query_url)
        # also requires '&darwin=false&destination=WAT&train_status=passenger'

        UkTransportSensor.__init__(
            self, sensor_name, api_app_id, api_app_key, query_url
        )

    def update(self):
        """Get the latest live departure data for the specified stop."""
        params = {'darwin': 'false', 'destination': self._destination_name, 'train_status': 'passenger'}  

        self._do_api_request(params)

        if self._data != {}:
            self._next_trains = []

            for departure in self._data['departures']['all']:      # don't need a regex search as passing in destination to search                
                #print_json(departure)   # uncomment to see all fields
                self._next_trains.append({
                    'origin_name': departure['origin_name'],
                    'destination_name': departure['destination_name'],
                    'status': departure['status'],
                    'scheduled': departure['aimed_departure_time'],                            
                    'estimated': departure['expected_departure_time'],
                    'platform': departure['platform'],
                    'operator_name': departure['operator_name']
                    })

            self._state = min(map(
                _delta_mins, [train['scheduled'] for train in self._next_trains]
            ))

    @property
    def device_state_attributes(self):
        """Return other details about the sensor state."""
        if self._data is not None:
            attrs = {ATTR_ATTRIBUTION: ATTRIBUTION}  # {'attribution': 'Data provided by transportapi.com'}
            for key in [
                    ATTR_ATCOCODE, ATTR_LOCALITY, ATTR_STOP_NAME,
                    ATTR_REQUEST_TIME
            ]:
                attrs[key] = self._data.get(key)           # place these attributes 
            attrs[ATTR_NEXT_BUSES] = self._next_buses      # not in 
            print_json(attrs)
            return attrs

    @property
    def unit_of_measurement(self):
        """Return the unit this state is expressed in."""
        return "min" 

In [142]:
station_code = 'WIM'
destination_name = 'WAT'

Train_sensor_1 = UkTransportLiveTrainTimeSensor(CONF_API_APP_ID, CONF_API_APP_KEY, station_code, destination_name)
Train_sensor_1.update()

train/station/WIM/live.json


In [145]:
Train_sensor_1._state

0

In [144]:
len(Train_sensor_1._data)

6